In [1]:
from PIL import Image
Image.CUBIC = Image.BICUBIC
import ttkbootstrap as ttk
from ttkbootstrap.constants import *
import tkinter as tk
import os, time, sys, queue, datetime
from csv import writer
from datetime import datetime, timedelta
from threading import Thread, Event
from tkinter import filedialog, Canvas, Label, LabelFrame, Frame, PhotoImage, Button, Entry, Scrollbar, StringVar, IntVar, DoubleVar#, ttk 
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure
import multiprocessing as mp
import http.server
import socketserver
from contextlib import closing
from more_itertools import time_limited

# root window (parent to all), controller to Frames
class Windows(tk.Tk):
    def __init__(self, *args, **kwargs):
        tk.Tk.__init__(self, *args, **kwargs)
        self.shared_data = {
            "geometry":    StringVar(self,"1280x800"),
            "padding":     IntVar(self, 5),
        }
        self.wm_title("Creative Envy Dashboard")
        ttk.Style("darkly")
        self.geometry(self.shared_data["geometry"].get())
        self.resizable(False, False)
        self.container = Frame(self)
        self.container.grid_rowconfigure(0, weight=1)
        self.container.grid_columnconfigure(0, weight=1)
        main_notebook = ttk.Notebook(
            self,
            height = str(int(int(self.shared_data["geometry"].get().split("x")[1]) - (self.shared_data["padding"].get() * 9))),
            width = str(int(int(self.shared_data["geometry"].get().split("x")[0]) - (self.shared_data["padding"].get() * 3))),
        ) # "self" as passed argument means Windows is parent
        for i, F in enumerate([Pane1, Pane2, Pane3]):
            frame = F(main_notebook, self)
            main_notebook.add(frame, text= 'Tab %s' % (i + 1)) # "self." adds to this instance, ...(self) adds to parent instance
        main_notebook.grid(
            row=0,
            column=0,
            #sticky="nsew",
            padx=(self.shared_data["padding"].get(), int(self.shared_data["geometry"].get().split("x")[0]) - self.shared_data["padding"].get()),
            pady=(self.shared_data["padding"].get(), int(self.shared_data["geometry"].get().split("x")[0]) - self.shared_data["padding"].get()),
        )
        
        def on_tab_change(event):
            index = main_notebook.index(main_notebook.select())
        main_notebook.bind("<<NotebookTabChanged>>", on_tab_change)

class Pane1(tk.Frame): # child to Notebook
    def __init__(self, parent, controller):
        tk.Frame.__init__(self, parent)
        self.controller = controller
        
        # Pane1 Objects
        # Time/Sunrise/Sunset
        lf1 = ttk.LabelFrame(
            self,
            height = str(int((int(self.controller.shared_data["geometry"].get().split("x")[0]) - (self.controller.shared_data["padding"].get() * 20)) / 5)),
            width = str(int((int(self.controller.shared_data["geometry"].get().split("x")[0]) - (self.controller.shared_data["padding"].get() * 20)) / 5)),
            relief="solid",
            borderwidth=1,
            padding=self.controller.shared_data["padding"].get(),
            text="Time/Sunrise/Sunset",
        ).grid(
            row=0,
            column=0,
            sticky="nsew",
            padx=(self.controller.shared_data["padding"].get(), self.controller.shared_data["padding"].get()),
            pady=(self.controller.shared_data["padding"].get(), self.controller.shared_data["padding"].get()),
            # ipadx=(self.controller.shared_data["padding"].get(), self.controller.shared_data["padding"].get()),
            # ipady=(self.controller.shared_data["padding"].get(), self.controller.shared_data["padding"].get()),
        )

        # Moon Phase
        lf2 = ttk.LabelFrame(
            self,
            height = str(int((int(self.controller.shared_data["geometry"].get().split("x")[0]) - (self.controller.shared_data["padding"].get() * 20)) / 5)),
            width = str(int((int(self.controller.shared_data["geometry"].get().split("x")[0]) - (self.controller.shared_data["padding"].get() * 20)) / 5)),
            relief="solid",
            borderwidth=1,
            padding=self.controller.shared_data["padding"].get(),
            text="Moon Phase",
        ).grid(row=0, column=1, sticky="nsew", padx=self.controller.shared_data["padding"].get(), pady=self.controller.shared_data["padding"].get())
        
        # Wx Forecast
        lf3 = ttk.LabelFrame(
            self,
            height = str(int((int(self.controller.shared_data["geometry"].get().split("x")[0]) - (self.controller.shared_data["padding"].get() * 20)) / 5)),
            width = str(int((int(self.controller.shared_data["geometry"].get().split("x")[0]) - (self.controller.shared_data["padding"].get() * 20)) / 5) * 2), # (pad value halved because width doubled)
            relief="solid",
            borderwidth=1,
            padding=self.controller.shared_data["padding"].get(),
            text="Wx Forecast",
        ).grid(row=0, column=2,  columnspan = 2, sticky="nsew", padx=self.controller.shared_data["padding"].get(), pady=self.controller.shared_data["padding"].get())
        
        # Extrapolated Honey Wt./Bee Count
        lf5 = ttk.LabelFrame(
            self,
            height = str(int((int(self.controller.shared_data["geometry"].get().split("x")[0]) - (self.controller.shared_data["padding"].get() * 20)) / 5)),
            width = str(int((int(self.controller.shared_data["geometry"].get().split("x")[0]) - (self.controller.shared_data["padding"].get() * 20)) / 5)),
            relief="solid",
            borderwidth=1,
            padding=self.controller.shared_data["padding"].get(),
            text="Honey Wt./Bee Count",
        ).grid(row=0, column=4, sticky="nsew", padx=self.controller.shared_data["padding"].get(), pady=self.controller.shared_data["padding"].get())
        
        # AWS Temp
        lf6 = ttk.LabelFrame(
            self,
            height = str(int((int(self.controller.shared_data["geometry"].get().split("x")[0]) - (self.controller.shared_data["padding"].get() * 20)) / 5)),
            width = str(int((int(self.controller.shared_data["geometry"].get().split("x")[0]) - (self.controller.shared_data["padding"].get() * 20)) / 5)),
            relief="solid",
            borderwidth=1,
            padding=self.controller.shared_data["padding"].get(),
            text="Temp",
        ).grid(row=1, column=0, sticky="nsew", padx=self.controller.shared_data["padding"].get(), pady=self.controller.shared_data["padding"].get())
        
        # AWS Solar
        lf7 = ttk.LabelFrame(
            self,
            height = str(int((int(self.controller.shared_data["geometry"].get().split("x")[0]) - (self.controller.shared_data["padding"].get() * 20)) / 5)),
            width = str(int((int(self.controller.shared_data["geometry"].get().split("x")[0]) - (self.controller.shared_data["padding"].get() * 20)) / 5)),
            relief="solid",
            borderwidth=1,
            padding=self.controller.shared_data["padding"].get(),
            text="Solar Rad.",
        ).grid(row=1, column=1, sticky="nsew", padx=self.controller.shared_data["padding"].get(), pady=self.controller.shared_data["padding"].get())
        
        # AWS Wind Spd/Dir
        lf8 = ttk.LabelFrame(
            self,
            height = str(int((int(self.controller.shared_data["geometry"].get().split("x")[0]) - (self.controller.shared_data["padding"].get() * 20)) / 5)),
            width = str(int((int(self.controller.shared_data["geometry"].get().split("x")[0]) - (self.controller.shared_data["padding"].get() * 20)) / 5)),
            relief="solid",
            borderwidth=1,
            padding=self.controller.shared_data["padding"].get(),
            text="Wind Spd./Dir.",
        ).grid(row=1, column=2, sticky="nsew", padx=self.controller.shared_data["padding"].get(), pady=self.controller.shared_data["padding"].get())
        
        # AWS Chicken Temp
        lf9 = ttk.LabelFrame(
            self,
            height = str(int((int(self.controller.shared_data["geometry"].get().split("x")[0]) - (self.controller.shared_data["padding"].get() * 20)) / 5)),
            width = str(int((int(self.controller.shared_data["geometry"].get().split("x")[0]) - (self.controller.shared_data["padding"].get() * 20)) / 5)),
            relief="solid",
            borderwidth=1,
            padding=self.controller.shared_data["padding"].get(),
            text="Chook Temp",
        ).grid(row=1, column=3, sticky="nsew", padx=self.controller.shared_data["padding"].get(), pady=self.controller.shared_data["padding"].get())
        
        # Bee Wt.
        lf10 = ttk.LabelFrame(
            self,
            height = str(int((int(self.controller.shared_data["geometry"].get().split("x")[0]) - (self.controller.shared_data["padding"].get() * 20)) / 5)),
            width = str(int((int(self.controller.shared_data["geometry"].get().split("x")[0]) - (self.controller.shared_data["padding"].get() * 20)) / 5)),
            relief="solid",
            borderwidth=1,
            padding=self.controller.shared_data["padding"].get(),
            text="Bee Wt.",
        ).grid(row=1, column=4, sticky="nsew", padx=self.controller.shared_data["padding"].get(), pady=self.controller.shared_data["padding"].get())
        
        # AWS Humid
        lf11 = ttk.LabelFrame(
            self,
            height = str(int((int(self.controller.shared_data["geometry"].get().split("x")[0]) - (self.controller.shared_data["padding"].get() * 20)) / 5)),
            width = str(int((int(self.controller.shared_data["geometry"].get().split("x")[0]) - (self.controller.shared_data["padding"].get() * 20)) / 5)),
            relief="solid",
            borderwidth=1,
            padding=self.controller.shared_data["padding"].get(),
            text="Humidity",
        ).grid(row=2, column=0, sticky="nsew", padx=self.controller.shared_data["padding"].get(), pady=self.controller.shared_data["padding"].get())
        
        # AWS UV Index
        lf12 = ttk.LabelFrame(
            self,
            height = str(int((int(self.controller.shared_data["geometry"].get().split("x")[0]) - (self.controller.shared_data["padding"].get() * 20)) / 5)),
            width = str(int((int(self.controller.shared_data["geometry"].get().split("x")[0]) - (self.controller.shared_data["padding"].get() * 20)) / 5)),
            relief="solid",
            borderwidth=1,
            padding=self.controller.shared_data["padding"].get(),
            text="UV Index",
        ).grid(row=2, column=1, sticky="nsew", padx=self.controller.shared_data["padding"].get(), pady=self.controller.shared_data["padding"].get())
        
        # AWS Precip
        lf13 = ttk.LabelFrame(
            self,
            height = str(int((int(self.controller.shared_data["geometry"].get().split("x")[0]) - (self.controller.shared_data["padding"].get() * 20)) / 5)),
            width = str(int((int(self.controller.shared_data["geometry"].get().split("x")[0]) - (self.controller.shared_data["padding"].get() * 20)) / 5)),
            relief="solid",
            borderwidth=1,
            padding=self.controller.shared_data["padding"].get(),
            text="Precip.",
        ).grid(row=2, column=2, sticky="nsew", padx=self.controller.shared_data["padding"].get(), pady=self.controller.shared_data["padding"].get())
        
        # Bee Temp
        lf14 = ttk.LabelFrame(
            self,
            height = str(int((int(self.controller.shared_data["geometry"].get().split("x")[0]) - (self.controller.shared_data["padding"].get() * 20)) / 5)),
            width = str(int((int(self.controller.shared_data["geometry"].get().split("x")[0]) - (self.controller.shared_data["padding"].get() * 20)) / 5)),
            relief="solid",
            borderwidth=1,
            padding=self.controller.shared_data["padding"].get(),
            text="Bee Temp.",
        ).grid(row=2, column=3, sticky="nsew", padx=self.controller.shared_data["padding"].get(), pady=self.controller.shared_data["padding"].get())
        
        # Bee Humid
        lf15 = ttk.LabelFrame(
            self,
            height = str(int((int(self.controller.shared_data["geometry"].get().split("x")[0]) - (self.controller.shared_data["padding"].get() * 20)) / 5)),
            width = str(int((int(self.controller.shared_data["geometry"].get().split("x")[0]) - (self.controller.shared_data["padding"].get() * 20)) / 5)),
            relief="solid",
            borderwidth=1,
            padding=self.controller.shared_data["padding"].get(),
            text="Bee Humid.",
        ).grid(row=2, column=4, sticky="nsew", padx=self.controller.shared_data["padding"].get(), pady=self.controller.shared_data["padding"].get())
        
        # AWS Temp
        meter1 = ttk.Meter(
            lf6,
            metersize=int(((int(self.controller.shared_data["geometry"].get().split("x")[0]) - (self.controller.shared_data["padding"].get() * 20)) / 5) * 0.8),
            padding=self.controller.shared_data["padding"].get(),
            amountused=25,
            metertype="semi",
            subtext="Temp",
            showtext=False,
            interactive=False,
        ).grid(
            row=1,
            column=0,
            sticky="nsew",
            padx=self.controller.shared_data["padding"].get(),
            pady=self.controller.shared_data["padding"].get(),
        )
        
#         # AWS Solar
#         meter2 = ttk.Meter(
#             parent,
#             metersize=int(((int(self.controller.shared_data["geometry"].get().split("x")[0]) - (self.controller.shared_data["padding"].get() * 20)) / 5) * 0.8),
#             amountused=65,
#             metertype="semi",
#             subtext="Solar Rad.",
#             showtext=False,
#             interactive=False,
#         ).grid(row=1, column=1, sticky="nsew", padx=self.controller.shared_data["padding"].get(), pady=self.controller.shared_data["padding"].get())
        
#         # AWS Wind Spd/Dir
#         meter3 = ttk.Meter(
#             parent,
#             metersize=int(((int(self.controller.shared_data["geometry"].get().split("x")[0]) - (self.controller.shared_data["padding"].get() * 20)) / 5) * 0.8),
#             amountused=25,
#             metertype="semi",
#             subtext="Wind Spd./Dir.",
#             showtext=False,
#             interactive=False,
#         ).grid(row=1, column=2, sticky="nsew", padx=self.controller.shared_data["padding"].get(), pady=self.controller.shared_data["padding"].get())
        
#         # AWS Chicken Temp
#         meter4 = ttk.Meter(
#             parent,
#             metersize=int(((int(self.controller.shared_data["geometry"].get().split("x")[0]) - (self.controller.shared_data["padding"].get() * 20)) / 5) * 0.8),
#             amountused=65,
#             metertype="semi",
#             subtext="Chook Temp",
#             showtext=False,
#             interactive=False,
#         ).grid(row=1, column=3, sticky="nsew", padx=self.controller.shared_data["padding"].get(), pady=self.controller.shared_data["padding"].get())
        
#         # Bee Wt.
#         meter5 = ttk.Meter(
#             parent,
#             metersize=int(((int(self.controller.shared_data["geometry"].get().split("x")[0]) - (self.controller.shared_data["padding"].get() * 20)) / 5) * 0.8),
#             amountused=25,
#             metertype="semi",
#             subtext="Bee Wt.",
#             showtext=False,
#             interactive=False,
#         ).grid(row=1, column=4, sticky="nsew", padx=self.controller.shared_data["padding"].get(), pady=self.controller.shared_data["padding"].get())
        
#         # AWS Humid
#         meter6 = ttk.Meter(
#             parent,
#             metersize=int(((int(self.controller.shared_data["geometry"].get().split("x")[0]) - (self.controller.shared_data["padding"].get() * 20)) / 5) * 0.8),
#             amountused=65,
#             metertype="semi",
#             subtext="Humidity",
#             showtext=False,
#             interactive=False,
#         ).grid(row=2, column=0, sticky="nsew", padx=self.controller.shared_data["padding"].get(), pady=self.controller.shared_data["padding"].get())
        
#         # AWS UV Index
#         meter7 = ttk.Meter(
#             parent,
#             metersize=int(((int(self.controller.shared_data["geometry"].get().split("x")[0]) - (self.controller.shared_data["padding"].get() * 20)) / 5) * 0.8),
#             amountused=25,
#             metertype="semi",
#             subtext="UV Index",
#             showtext=False,
#             interactive=False,
#         ).grid(row=2, column=1, sticky="nsew", padx=self.controller.shared_data["padding"].get(), pady=self.controller.shared_data["padding"].get())

#         # AWS Precip
#         meter8 = ttk.Meter(
#             parent,
#             metersize=int(((int(self.controller.shared_data["geometry"].get().split("x")[0]) - (self.controller.shared_data["padding"].get() * 20)) / 5) * 0.8),
#             amountused=65,
#             metertype="semi",
#             subtext="Precip.",
#             showtext=False,
#             interactive=False,
#         ).grid(row=2, column=2, sticky="nsew", padx=self.controller.shared_data["padding"].get(), pady=self.controller.shared_data["padding"].get())
        
#         # Bee Temp
#         meter9 = ttk.Meter(
#             parent,
#             metersize=int(((int(self.controller.shared_data["geometry"].get().split("x")[0]) - (self.controller.shared_data["padding"].get() * 20)) / 5) * 0.8),
#             amountused=25,
#             metertype="semi",
#             subtext="Bee Temp.",
#             showtext=False,
#             interactive=False,
#         ).grid(row=2, column=3, sticky="nsew", padx=self.controller.shared_data["padding"].get(), pady=self.controller.shared_data["padding"].get())
        
#         # Bee Humid
#         meter10 = ttk.Meter(
#             parent,
#             metersize=int(((int(self.controller.shared_data["geometry"].get().split("x")[0]) - (self.controller.shared_data["padding"].get() * 20)) / 5) * 0.8),
#             amountused=65,
#             metertype="semi",
#             subtext="Bee Humid.",
#             showtext=False,
#             interactive=False,
#         ).grid(row=2, column=4, sticky="nsew", padx=self.controller.shared_data["padding"].get(), pady=self.controller.shared_data["padding"].get())

class Pane2(tk.Frame):
    def __init__(self, parent, controller):
        tk.Frame.__init__(self, parent)
        self.controller = controller
        # Pane2 Objects
        canvas1 = tk.Canvas(
            self,
            width = str(int(self.controller.shared_data["geometry"].get().split("x")[0])),
            height = str(int(self.controller.shared_data["geometry"].get().split("x")[1])),
        ).grid(row=0, column=0)

class Pane3(tk.Frame): # Picture Frame (need to install pyicloud)
    def __init__(self, parent, controller):
        tk.Frame.__init__(self, parent)
        self.controller = controller
        # Pane3 Objects
        canvas1 = tk.Canvas(
            self,
            width = str(int(self.controller.shared_data["geometry"].get().split("x")[0])),
            height = str(int(self.controller.shared_data["geometry"].get().split("x")[1])),
        )
        canvas1.create_rectangle(0, 0, int(self.controller.shared_data["geometry"].get().split("x")[0]), int(self.controller.shared_data["geometry"].get().split("x")[1]), fill="#444444")
        canvas1.create_rectangle(20, 20, int(self.controller.shared_data["geometry"].get().split("x")[0]) - 20, int(self.controller.shared_data["geometry"].get().split("x")[1]) - 50, fill="black")
        canvas1.grid(row=0, column=0)

def main(): # creates main window and calls main loop
    windows = Windows()
    windows.mainloop()
    
if __name__ == '__main__': # runs main if in python script
    main()